<h2 style = "background-color:SandyBrown; border: 1px solid black; text-align: center; "> 
    <br>In this notebook I will focus on movies Genres as an independant variables  <br>
    

<ul style = "background-color:pink; border: 1px solid black; font-size:150%;"> 
    <li> EDA for the Genres (Independant Varaible)
     <li> Clustring and building a recommended movies based on Genres

In [ ]:
#import the basic liberaries for EDA
import numpy as np
import pandas as pd
import ast
from collections import Counter

#Plotting liberaries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import missingno as msno #https://github.com/ResidentMario/missingno

# upsupervised liberaries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Stop warnings
import warnings
warnings.filterwarnings("ignore")

#read the df
df = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
#print sample of the Data frame
df.head(2)

<h2 style = "background-color:gray; border: 1px solid black; text-align: center; "> 
    <br>Let's check the missing value using missingno liberary<br>
    

In [ ]:
msno.matrix(df); #quickly visually pick out patterns in data completion.

### The missing values in the home page and tagline will not effect my analysis 

<h2 style = "background-color:powderblue; border: 1px solid black; text-align: center; "> 
    <br>First: let's handel the pandas columns contain list[  ] of dicts{ } <br>
    


## Print sample of Genres columns 

In [ ]:
print ('Sample of Geners Column\n'  , df.genres[0])
print ("_"*40)

## Create the Conversion function

In [ ]:
# Cridet for this function goes to this notebook
# https://www.kaggle.com/yashsaxena17/movie-recommendation
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

## Print the Final output

In [ ]:
df['genres'] = df['genres'].apply(convert)
print ('Sample of Geners Column\n'  , df.genres[0])
print ("_"*40)

<h2 style = "background-color:powderblue; border: 1px solid black; text-align: center; "> 
    <br>Generate a new Data frame <br> <br>Independant Variable : Genres.<br>

## Get the frequency in Counter() and convert it to dataframe

In [ ]:
geners = Counter()
for i in range(df.shape[0]):
    for j in df.genres[i]:
        geners[j]+=1
Geners_df = pd.DataFrame.from_dict(geners, orient='index').reset_index()
Geners_df = Geners_df.rename(columns = {'index': 'Geners' ,0: 'Frequency'})
Geners_df.head(10)

## Map the the dependant variables average of: <br><br>(vote_count , vote_average , Popularity , budget , revenue ) <br><br>from the Original df to the genres df

In [ ]:
generes_list = Geners_df.Geners.tolist()

Geners_Vote_Average = {}
Geners_Vote_Count = {}
Geners_Popularity = {}
Geners_budget = {}
Geners_revenue = {}



for j in generes_list:
    gener = []
    for i in range (df.shape[0]):
        if j in  df.genres[i]:
            gener.append(df.vote_average[i])
            Geners_Vote_Average[j] = sum(gener)/len(gener)
            
for j in generes_list:
    gener = []
    for i in range (df.shape[0]):
        if j in  df.genres[i]:
            gener.append(df.vote_count[i])
            Geners_Vote_Count[j] = sum(gener)/len(gener)
            
for j in generes_list:
    gener = []
    for i in range (df.shape[0]):
        if j in  df.genres[i]:
            gener.append(df.popularity[i])
            Geners_Popularity[j] = sum(gener)/len(gener)

for j in generes_list:
    gener = []
    for i in range (df.shape[0]):
        if j in  df.genres[i]:
            gener.append(df.budget[i])
            Geners_budget[j] = sum(gener)/len(gener)

            
for j in generes_list:
    gener = []
    for i in range (df.shape[0]):
        if j in  df.genres[i]:
            gener.append(df.revenue[i])
            Geners_revenue[j] = sum(gener)/len(gener)

                                              


                                              
Geners_df['Average_Vote_Average'] = Geners_df.Geners.map(Geners_Vote_Average)
Geners_df['Average_Vote_Count'] = Geners_df.Geners.map(Geners_Vote_Count)
Geners_df['Average_Popularity'] = Geners_df.Geners.map(Geners_Popularity)
Geners_df['Average_budget'] = Geners_df.Geners.map(Geners_budget)
Geners_df['Average_revenue'] = Geners_df.Geners.map(Geners_revenue)
# Round the Budget and revenue to the million
Geners_df['Average_budget'] = Geners_df['Average_budget']/1000000
Geners_df['Average_revenue']  = Geners_df['Average_revenue'] /1000000

In [ ]:
Geners_df.style.background_gradient(cmap='YlGnBu')

<h2 style = "background-color:powderblue; border: 1px solid black; text-align: center; "> 
    <br>Let's visualize the final Genres dataframe<br>

In [ ]:
# Create dependant variables list to use it in for loop
dependant_variables = ['Frequency', 'Average_Vote_Average', 'Average_Vote_Count',
                       'Average_Popularity', 'Average_budget', 'Average_revenue']
#Set the size of the horizontal bars
h = np.array([0.4,0.4 ,0.4 ,0.4 ,0.4 ,0.4,0.4 ,0.4,
             0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.7,0.8,1])
#Set the Colors of the bars
c = np.array(['lightgray','lightgray' ,'lightgray' ,'lightgray' ,'lightgray' ,'lightgray',
              'lightgray' ,'lightgray',
             'lightgray','lightgray','lightgray'
              ,'lightgray','lightgray','lightgray','lightgray',
              'lightgray','lightgray','gray','tomato','gold'])
#Create the figure and the axis 
fig , ax = plt.subplots(nrows = 4, ncols = 2 , figsize = [12,30])
#set the figure background color
fig.patch.set_facecolor('#edf2fa')
#Figure Title
fig.suptitle('\n\nGenres Ranking Across Dependant Variables', fontfamily='serif',
                   fontsize = 24 , fontweight='bold' )
#for loop (axis and variable ) to generate the barh plot 
for axes , variable in zip (ax.flatten() , dependant_variables):
    #Sorting the Genres dataframe by virables
    Geners_df = Geners_df.sort_values(by = variable )
    #Barh plot using Matplotlib
    axes.barh(Geners_df.Geners ,Geners_df[variable] , height = h  , color = c)
    #set the axes background color
    axes.set_facecolor('#edf2fa')
    #set the x limit
    mx = Geners_df[variable].max()*1.3
    axes.set_xlim(0 , mx)
    #Create a background diffrent color for the top three Genres
    axes.axhspan(ymin = 16.3 , ymax = 19.7 , fc='#ff73d5', alpha=0.2 , zorder=0)
    #remove the Axes Boundary except the Left side
    for s in ['top', 'right', 'bottom']:
         axes.spines[s].set_visible(False)
    #add the notation on the bars
    for patch in axes.patches:
        try:
            #try exepct technique was used becouse the axhspan has no attrebuite get_width nor get_height
            bl = patch.get_xy()
            x = 1.05 * patch.get_width() + bl[0]
            y = 0.5 * patch.get_height() + bl[1] 
            axes.text(x,y,"%d" %(patch.get_width()),va='center',ha='center' , 
                      color = 'black',fontsize = 12, fontfamily='serif')
        except:
            #skip and go to next patch
            continue
    #Get it more clean and remove the useless ticks and labels
    axes.set_xticks([])
    axes.set_xlabel("")
    axes.set_ylabel("")
    axes.set_title(f"Sorting of Genres according to {variable}" , fontfamily='serif',
                   fontsize = 12 , fontweight='bold')
    #End of for loop
    

#Add some text on the visualization 
ax[0,0].text(1270, 8, 'Drama ', fontsize=18, fontfamily='serif', color = 'Gold')
ax[0,0].text(750, 7, 'is the most genres produced ', fontsize=12, fontfamily='serif', color = '#ff73d5')
ax[0,0].text(750, 6, 'Next was Comedy and Thriller', fontsize=12,  fontfamily='serif', color = '#ff73d5');


ax[1,1].text(26, 6, 'Adventure',  fontsize=18,   fontfamily='serif', color = 'Gold')
ax[1,1].text(22, 5,  'stays on the top for both ', fontsize=12,  fontfamily='serif', color = '#ff73d5')
ax[1,1].text(22, 4, 'Popularity and vote counts', fontsize=12,  fontfamily='serif', color = '#ff73d5');


ax[2,0].text(30, 2,  'Animation , Adventure\n     and Fantasy', fontsize=18,  fontfamily='serif', color = 'Gold')
ax[2,0].text(40, 0,  'Highest Budget \nHighest revenue ', fontsize=14,  fontfamily='serif', color = '#ff73d5');

#lets add some arrows
#https://stackoverflow.com/questions/44526103/matplotlib-draw-curved-arrow-that-looks-just-like-pyplot-arrow
style = "Simple, tail_width=0.5, head_width=6, head_length=9"
kw = dict(arrowstyle=style, color="Gold")
a = patches.FancyArrowPatch((1870, 8.17), (2310, 18.9),connectionstyle="arc3,rad=.7", **kw)
ax[0,0].add_patch(a)
a2 = patches.FancyArrowPatch((42, 5.30), (40, 18.9),connectionstyle="arc3,rad=.2", **kw)
ax[1,1].add_patch(a2)
a3 = patches.FancyArrowPatch((60 , 4), (66, 18.9),connectionstyle="arc3,rad=.2", **kw)
ax[2,0].add_patch(a3)

# Create the arrow across two supblots
#https://www.cilyan.org/blog/2016/01/23/matplotlib-draw-between-subplots/
# 1. Get transformation operators for axis and figure
ax0tr = ax[1,0].transData # Axis 0 -> Display
ax1tr = ax[1,1].transData # Axis 1 -> Display
figtr = fig.transFigure.inverted() # Display -> Figure
# 2. Transform arrow start point from axis 0 to figure coordinates
ptE = figtr.transform(ax0tr.transform((1430, 0.047)))
# 3. Transform arrow end point from axis 1 to figure coordinates
ptB = figtr.transform(ax1tr.transform((42, -0.018)))
# Place arrow in figure coord system
arrow = patches.FancyArrowPatch(ptB, ptE, transform=fig.transFigure, connectionstyle="arc3,rad=0.2" , **kw)
# 5. Add patch to list of objects to draw onto the figure
fig.patches.append(arrow)


# 1. Get transformation operators for axis and figure
ax0tr = ax[2,0].transData # Axis 0 -> Display
ax1tr = ax[2,1].transData # Axis 1 -> Display
figtr = fig.transFigure.inverted() # Display -> Figure
# 2. Transform arrow start point from axis 0 to figure coordinates
ptB = figtr.transform(ax0tr.transform((63, -.03)))
# 3. Transform arrow end point from axis 1 to figure coordinates
ptE = figtr.transform(ax1tr.transform((227, 0.048)))
# Place arrow in figure coord system
arrow2 = patches.FancyArrowPatch(ptB, ptE, transform=fig.transFigure, connectionstyle="arc3,rad=0.2" , **kw)
# 5. Add patch to list of objects to draw onto the figure
fig.patches.append(arrow2)

# adding the scatter Plots

sns.scatterplot(x = Geners_df.Average_Vote_Count , y = Geners_df.Average_Popularity  , 
           s = Geners_df.Average_budget*5 , hue =Geners_df.Geners , ax = ax[3,0] ,
                style=Geners_df.Geners , legend = False , palette="Dark2")
ax[3,0].set_facecolor('#edf2fa')
for s in ['top', 'right']:
    ax[3,0].spines[s].set_visible(False)
ax[3,0].set_xticks([])
ax[3,0].set_yticks([])
ax[3,0].set_xlabel("Average Vote Counts" ,fontsize=14,   fontfamily='serif')
ax[3,0].set_ylabel("Average Popularity" , fontsize=14,   fontfamily='serif')
ax[3,0].text( -200,45 ,"Plotting Scatter to find the Corelations Between Dependant Varaible" , fontfamily='serif',
                   fontsize = 18 , fontweight='bold')
    
    
    
sns.scatterplot(x = Geners_df.Average_budget , y = Geners_df.Average_revenue  , 
           s = Geners_df.Average_Popularity*5 , hue =Geners_df.Geners , ax = ax[3,1] ,
                style=Geners_df.Geners , legend = "full" , palette="Dark2")
plt.legend(bbox_to_anchor=(-1.40, -.20), loc="lower left", borderaxespad=0. , ncol=8 , 
          facecolor='#edf2fa' , framealpha=.01)
ax[3,1].set_facecolor('#edf2fa')

for s in ['top', 'right']:
    ax[3,1].spines[s].set_visible(False)
ax[3,1].set_xticks([])
ax[3,1].set_yticks([])
ax[3,1].set_xlabel("Average_budget" ,fontsize=14,   fontfamily='serif')
ax[3,1].set_ylabel("Average_revenue" , fontsize=14,   fontfamily='serif');


ax[3,0].add_patch(plt.Rectangle((1200, 35), 600 , 50, color='#ff73d5', alpha=0.2 , zorder = 0))


style = "Simple, tail_width=0.5, head_width=6, head_length=9"
kw = dict(arrowstyle=style, color="Gold")
ax[3,0].add_patch(patches.FancyArrowPatch((800, 38), (1200, 38) , **kw))
ax[3,0].text(50, 36,  'Adventure , Animation\nFantasy , ScienceFiction', fontsize=18,  fontfamily='serif', color = 'Gold')
                  

    
    
    
    
ax[3,1].add_patch(plt.Rectangle((60, 190),10 , 50, color='#ff73d5', alpha=0.2 , zorder = 0))


style = "Simple, tail_width=0.5, head_width=6, head_length=9"
kw = dict(arrowstyle=style, color="Gold")
ax[3,1].add_patch(patches.FancyArrowPatch((30, 200), (60, 200) , **kw))
ax[3,1].text(5, 200,  'Animation , Adventure\n       Fantasy', fontsize=18,  fontfamily='serif', color = 'Gold')
  
    
    
    
##https://matplotlib.org/stable/gallery/subplots_axes_and_figures/demo_constrained_layout.html
## To Be Continue


<h2 style = "background-color:powderblue; border: 1px solid black; text-align: center; "> 
    <br>Conclusion of Genres Analysis<br>

<strong><ul style = "background-color:tomato; border: 1px solid black; "> 
    <li> Most produced genres in the list : [Drama / Comedy]
        <li> Most Vote counts goes to : [Adventure /Science Fiction]
            <li> Highest popularity : [Adventure / Animation]
                <li> Highest Budget : [Animation / Advenure]
                    <li> Highest Revenue : [Animation / Advenure]

<h2 style = "background-color:powderblue; border: 1px solid black; text-align: center; "> 
    <br>Unsupervised Task - Clustring movies based on the Genres only <br><br>
    Building a recommended system based on Genres<br><br>

In [ ]:
#Building new data frame
geners_list = Geners_df.Geners.tolist()
df_new = df[['original_title' , 'genres' , 'popularity']]
df_new[geners_list] =0 

In [ ]:
#Get dummies for the genres
for i in range (df_new.shape[0]):
    for j in  geners_list:
        if j in df.genres[i]:
            df_new[j] [i] = 1

In [ ]:
#decomposing the features in 2d
X = df_new.iloc[: , 3:].values
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

In [ ]:
#clustring using Kmean  - sklearn
model = KMeans(n_clusters=150)
model.fit(principalComponents)

labels = model.labels_

df_new['label'] = pd.DataFrame({'label':labels})

In [ ]:
#Build the Recommendation Function
def recommend_10 (film ):
    global df_new
    df_new = df_new.sort_values(by = 'popularity' , ascending = False)
    mark = df_new.loc[df_new['original_title'] == film, 'label'].iloc[0]
    print ("Label number : "  , mark)
    print ("_"*40)
    print ("\nRecommended movies for {} : ".format(film))
    print ("_"*40)
    lst = df_new[df_new.label == mark]['original_title'][0:10].tolist()
    for n , l in zip (range (1,11) , lst):
        print ( n , " : " , l)

In [ ]:
recommend_10 ('Mission: Impossible III' )

In [ ]:
recommend_10 ('Minions' )

In [ ]:
recommend_10 ('The Lord of the Rings: The Fellowship of the Ring' )

In [ ]:
recommend_10 ('Titanic' )

In [ ]:
recommend_10 ('Despicable Me' )

<h2 style = "background-color:Tan; border: 1px solid black; text-align: center; "> 
    <br>Thank you for visiting my notebook<br>